In [20]:
from dotenv import load_dotenv
load_dotenv()
import pandas as pd
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_agentchat.messages import TextMessage
from autogen_core import CancellationToken
import gradio as gr


In [27]:
df = pd.read_csv("vm_data.csv", parse_dates=['KN_SNC_DT', 'Decom_date'], date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d", errors='coerce'))

ReadDataSet = LangChainToolAdapter(PythonAstREPLTool(locals={"df": df}))
model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")


DataLoaderAgent = AssistantAgent(
    "assistant", tools=[ReadDataSet], model_client=model_client,system_message= """Use the `df` variable to access the dataset.
    Below data is metadata for df this data belongs to AppBrewery.
    Below is the description of df dataset.
    1. VMNAME -> Name of the virtual machine.
    2. State -> Indicates whether the VM is ON or OFF.
    3. Status -> Describes the VM's condition (Normal or Special).
    4. Host -> The physical server hosting the VM.
    5. Cluster -> The group of hosts managing the VM.
    6. Provisioned_Space -> Total storage allocated to the VM.
    7. Used_Space -> Storage currently used by the VM.
    8. HostCPU -> CPU allocation on the host machine.
    9. HostMem -> Memory allocated to the VM.
    10. KN_SNC_DT -> Known since date of the VM record.
    11. Decom_date -> Planned decommissioning date of the VM.
    """
)

/var/folders/p0/j24845kj1qs3ncsrk58vfq880000gn/T/ipykernel_17619/2699700280.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  df = pd.read_csv("vm_data.csv", parse_dates=['KN_SNC_DT', 'Decom_date'], date_parser=lambda x: pd.to_datetime(x, format="%Y%m%d", errors='coerce'))


In [28]:
response = await(
    DataLoaderAgent.on_messages(
        [TextMessage(content="What's the Trend for decomission date of VMs? Reply only in increasing or decreasing?", source="user")], CancellationToken()
    ))

In [32]:
response.chat_message.content

"NameError: name 'pd' is not defined"

In [30]:

import asyncio

def chat(question, history):
    response =  asyncio.run(
    DataLoaderAgent.on_messages(
        [TextMessage(content=question, source="user")], CancellationToken()
    ))
    return response.chat_message.content

In [31]:
view = gr.ChatInterface(chat, type="messages").launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7865

To create a public link, set `share=True` in `launch()`.


In [ ]:
await(Console(
    DataLoaderAgent.on_messages_stream(
        [TextMessage(content="What's the Trend for decomission date of VMs? Reply only in increasing or decreasing?", source="user")], CancellationToken()
    )
))

In [10]:
await(Console(
    DataLoaderAgent.on_messages_stream(
        [TextMessage(content="How many VMs under cluster EAPSIPVXT9753", source="user")], CancellationToken()
    )
))

---------- assistant ----------
[FunctionCall(id='call_D6ykPO4xpzJx4e4WkiXsI8uD', arguments='{"query":"vm_count_eapsipvxt9753 = df[df[\'Cluster\'] == \'EAPSIPVXT9753\'].shape[0]\\nvm_count_eapsipvxt9753"}', name='python_repl_ast')]
---------- assistant ----------
[FunctionExecutionResult(content='38', call_id='call_D6ykPO4xpzJx4e4WkiXsI8uD')]
---------- assistant ----------
38


Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, content='38', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=453, completion_tokens=53), content=[FunctionCall(id='call_D6ykPO4xpzJx4e4WkiXsI8uD', arguments='{"query":"vm_count_eapsipvxt9753 = df[df[\'Cluster\'] == \'EAPSIPVXT9753\'].shape[0]\\nvm_count_eapsipvxt9753"}', name='python_repl_ast')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='38', call_id='call_D6ykPO4xpzJx4e4WkiXsI8uD')], type='ToolCallExecutionEvent')])

In [11]:
await(Console(
    DataLoaderAgent.on_messages_stream(
        [TextMessage(content="How many VMs will be decomissioned by february this year?", source="user")], CancellationToken()
    )
))

---------- assistant ----------
[FunctionCall(id='call_vXc2qmza8J7SE0CLvD2Nvq6e', arguments='{"query":"import datetime\\n\\n# Current year\\ncurrent_year = datetime.datetime.now().year\\n\\n# Filter VMs with Decom_date until February this year\\ncount_decom_feb = df[(df[\'Decom_date\'] <= datetime.datetime(current_year, 2, 28))].shape[0]\\ncount_decom_feb"}', name='python_repl_ast')]
---------- assistant ----------
[FunctionExecutionResult(content='28', call_id='call_vXc2qmza8J7SE0CLvD2Nvq6e')]
---------- assistant ----------
28


Response(chat_message=ToolCallSummaryMessage(source='assistant', models_usage=None, content='28', type='ToolCallSummaryMessage'), inner_messages=[ToolCallRequestEvent(source='assistant', models_usage=RequestUsage(prompt_tokens=538, completion_tokens=84), content=[FunctionCall(id='call_vXc2qmza8J7SE0CLvD2Nvq6e', arguments='{"query":"import datetime\\n\\n# Current year\\ncurrent_year = datetime.datetime.now().year\\n\\n# Filter VMs with Decom_date until February this year\\ncount_decom_feb = df[(df[\'Decom_date\'] <= datetime.datetime(current_year, 2, 28))].shape[0]\\ncount_decom_feb"}', name='python_repl_ast')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant', models_usage=None, content=[FunctionExecutionResult(content='28', call_id='call_vXc2qmza8J7SE0CLvD2Nvq6e')], type='ToolCallExecutionEvent')])

In [ ]:

def chat(question, history):
    response =  asyncio.run(
    assistant_agent.on_messages(
        [TextMessage(content=question, source="user")], CancellationToken()
    ))
    return response.chat_message.content
    if isinstance(response.chat_message.content, dict):
        return response.chat_message.content.get("answer")
        return response.chat_message.content